In [3]:
#imports
import numpy as np
import pandas as pd
import matplotlib as plt
import os


In [5]:
#set working directory
wd = os.path.dirname(os.getcwd())
os.chdir(wd)
print("Working Directory: ", os.getcwd())

Working Directory:  c:\Users\Stephan\Documents\Python\Python-Stat\Neuer\Kaggle_semiinar\student-performance


In [ ]:
labels = pd.read_csv('data/raw/train_labels.csv')

In [ ]:
labels['session'] = labels.session_id.apply(lambda x: int(x.split('_')[0]) )
labels['q'] = labels.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )

In [ ]:
#deleting the now obsolete labels of the labels
labels= labels.set_index('session')
labels = labels.drop(["session_id"], axis= 1)


In [ ]:
#saving the dataset



labels.to_csv("data/processed/labels.csv",index_label= "session")

The Dataframe labels is saved. 
it contains the cols:
 session : the session id
 correct : 0 and 1 for the answer of the question
 q: number of the question from 1 to 18

The Data are stored as a .csv  

to load the data efficiently use this loading function



In [7]:
dtypes_labels= {
    'correct': np.uint8, 
    'q':np.uint8}

labels = pd.read_csv('data/processed/labels_q1-3.csv', dtype=dtypes_labels)

In [1]:
def load_train_data(file_path: str , dtypes: dict = None, n_rows: int = None):
    # If dtypes is not specified, set default data types for each column
    if dtypes is None:
        dtypes = {
            'level': np.uint8,  
            'session_id': np.int64,
            'level_group': 'category',
            'event_name': np.uint8,
            'name': np.uint8,
            'fqid': np.uint8,
            'room_fqid': np.uint8,
            'text_fqid': np.uint8,
            'fullscreen': np.uint8,
            'hq': np.uint8,
            'music': np.uint8,
            'hover_duration_mean': np.float32,
            'difference_clicks_mean': np.float32,
            "distance_clicks_mean": np.float32,
            "screen_distance_clicks_mean": np.float32,            
            'elapsed_time_std': np.float32,
            'page_std': np.float32,
            'room_coor_x_std': np.float32,
            'room_coor_y_std': np.float32,
            'screen_coor_x_std': np.float32,
            'screen_coor_y_std': np.float32,
            'hover_duration_std': np.float32,
            'difference_clicks_std': np.float32,
            "distance_clicks_std": np.float32,
            "screen_distance_clicks_std": np.float32,
            'index_sum_of_actions': np.int32,
            'difference_clicks_max': np.float32,
            'elapsed_time_max': np.float32,
            'clicks_per_second': np.float32,
            "sum_distance_clicks_max": np.float32,
        }
        
    # Read in the CSV file
    if n_rows is None:
        df = pd.read_csv(file_path, dtype=dtypes, index_col = 0)
    else:
        df = pd.read_csv(file_path, dtype=dtypes, nrows=n_rows, index_col= 0)
    
    # Set data types for columns with "_i" index in their name
    row, cols = df.shape
    if cols > 50:
        for column in df.columns:
            base_name = column.rsplit('_', 1)[0]  # get the base name by splitting on the last "_" character
            if base_name in dtypes:
                column_number = column.rsplit('_', 1)[1]  # get the number from the index by splitting on the last "_" character
                new_column_name = f"{base_name}_{column_number}"  # construct the new column name
                column_dtype = dtypes[base_name]
                df[new_column_name] = df[column].astype(column_dtype)  # set the same data type for all columns with the same base name

    return df

In [6]:
train_data = load_train_data("data/processed/df_0_4_flattened.csv")

In [19]:
labels_q2 = labels[labels["q"] == 2]
labels_q2.rename(columns= {"session": "session_id_1"}, inplace= True)

C:\Users\Stephan\AppData\Local\Temp\ipykernel_10904\2422067669.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels_q2.rename(columns= {"session": "session_id_1"}, inplace= True)


In [32]:
merged_df = pd.merge(labels_q2, train_data, on='session_id_1')


In [33]:
y = merged_df["correct"]
X = merged_df.drop(columns=["session_id_1", "level_group_1", "q", "correct"])


In [34]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Assuming you have a matrix of features called "X" (shape: [n_samples, 111]) and
# a target column called "correct" (shape: [n_samples]).

# Define the number of input features
input_dim = 107

# Define the neural network architecture
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=input_dim))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=10, batch_size=32)

# Make predictions
predictions = model.predict(X)

# Print predictions
print(predictions)


Epoch 1/10
737/737 [==============================] - 7s 6ms/step - loss: 21366600368128.0000 - accuracy: 0.9570
Epoch 2/10
737/737 [==============================] - 4s 5ms/step - loss: 17725309059072.0000 - accuracy: 0.9581
Epoch 3/10
737/737 [==============================] - 4s 5ms/step - loss: 13866495901696.0000 - accuracy: 0.9580
Epoch 4/10
737/737 [==============================] - 3s 4ms/step - loss: 7615312035840.0000 - accuracy: 0.9593
Epoch 5/10
737/737 [==============================] - 3s 5ms/step - loss: 5237923381248.0000 - accuracy: 0.9580
Epoch 6/10
737/737 [==============================] - 3s 4ms/step - loss: 4276999684096.0000 - accuracy: 0.9580
Epoch 7/10
737/737 [==============================] - 4s 6ms/step - loss: 3256495636480.0000 - accuracy: 0.9579
Epoch 8/10
737/737 [==============================] - 4s 6ms/step - loss: 2474828627968.0000 - accuracy: 0.9582
Epoch 9/10
737/737 [==============================] - 4s 6ms/step - loss: 3916167905280.0000 - accura